In [2]:
import pandas as pd
import os
import numpy as np
from tqdm import tqdm
import re

In [3]:
os.listdir("NOAA_Storm_events_dataset")

['README',
 'Storm-Data-Bulk-csv-Format.pdf',
 'Storm-Data-Export-Format.pdf',
 'StormEvents_details-ftp_v1.0_d1950_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1951_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1952_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1953_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1954_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1955_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1956_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1957_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1958_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1959_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1960_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1961_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1962_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1963_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1964_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1965_c20210803.csv',
 'StormEvents_details-ftp_v1.0_d1966_c20210803.csv',
 'StormEvents_detail

In [6]:
inflation_traj = pd.read_csv("Prod_datasets/inflation_traj_US.csv")


def clean_detailed_dataset(data):
    data.drop(columns = ["BEGIN_YEARMONTH", "BEGIN_DAY", "BEGIN_TIME", "END_YEARMONTH", "END_DAY", "END_TIME"], inplace = True)
    data["BEGIN_DATE_TIME"] = pd.to_datetime(data["BEGIN_DATE_TIME"], format = "mixed")
    data["END_DATE_TIME"] = pd.to_datetime(data["END_DATE_TIME"], format = "mixed")

    def convert_amounts(col_name, df = data):
        df[col_name] = df[col_name].astype(str)
        df[col_name] = df[col_name].apply(lambda x: re.sub(r"[^\w\s\.]", "", x))
        amount = df[col_name].str.split("(K|M|B|H|k|m|b|h)", regex = True, expand = True)
        print(amount)
        amount.replace(to_replace = [None, "", np.nan, "nan"], value = 0, inplace = True)

        def abreviation(val):
            if type(val) != type("str") or val != val: #Nan values
                return 0
            if val.lower() == "h":
                return 1e2
            if val.lower() == "k":
                return 1e3
            if val.lower() == "m":
                return 1e6
            if val.lower() == "b":
                return 1e9
        
        def force_convert_float(val):
            try:
                return float(val)
            except:
                return 0
            
        print(amount[0])

        if amount.shape[1] > 1:
            amount[1] = amount[1].apply(abreviation)

            amount[0] = amount[0].apply(force_convert_float)
            amount[1] = pd.to_numeric(amount[1])

            return(amount[0] * amount[1])
        
        else:
            return(pd.to_numeric(amount[0]))
        
    data['DAMAGE_PROPERTY'] = convert_amounts("DAMAGE_PROPERTY")
    data['DAMAGE_CROPS'] = convert_amounts("DAMAGE_CROPS")


    data = data.join(inflation_traj.set_index("YEAR"), on = "YEAR")

    return(data)

   


def clean_fatalities_dataset(data):
    data.drop(columns = ["FAT_YEARMONTH", "FAT_DAY", "FAT_TIME", "EVENT_YEARMONTH"], inplace = True)

    data["FATALITY_DATE"] = pd.to_datetime(data["FATALITY_DATE"], format = "mixed")
    
    return(data)

In [7]:
%%capture --no-stderr

path = "NOAA_Storm_events_dataset"

for file_name in tqdm(os.listdir(path)):
    print(file_name)
    _, ext = os.path.splitext(path+"/"+file_name)
    if ext == ".csv":
        data = pd.read_csv(path+"/"+file_name)
        if "details" in file_name:
            data = clean_detailed_dataset(data)
            name = "StormEvents_details_" + file_name[29:44]
        elif "fatalities" in file_name:
            data = clean_fatalities_dataset(data)
            name = "StormEvents_fatalities_" + file_name[32:47]
        elif "locations" in file_name:
            data = data
            name = "StormEvents_locations_" + file_name[31:46]
        else:
            print("Unknown dataset type: {}".format(file_name))

        data.to_csv("NOAA_Storm_events_clean/"+name+".csv")      

 23%|██▎       | 48/207 [00:55<07:36,  2.87s/it]C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_28392\2289264922.py:7: DtypeWarning: Columns (26,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path+"/"+file_name)
 24%|██▎       | 49/207 [01:00<09:14,  3.51s/it]C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_28392\2289264922.py:7: DtypeWarning: Columns (26,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path+"/"+file_name)
 24%|██▍       | 50/207 [01:06<11:27,  4.38s/it]C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_28392\2289264922.py:7: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path+"/"+file_name)
 25%|██▍       | 51/207 [01:16<15:42,  6.04s/it]C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_28392\2289264922.py:7: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set l